<a href="https://colab.research.google.com/github/pihlnikl/Data-analysis/blob/master/pihlnikl_decision_trees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# based on https://github.com/joelgrus/data-science-from-scratch/blob/master/first-edition/code/decision_trees.py
# adapted to python 3 

from collections import Counter, defaultdict
from functools import partial
import math
from pprint import pprint as pp
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None

In [3]:
def entropy(labeled_data):        
    labels = [label for _, label in labeled_data]
    class_probabilities = [count / len(labels) 
            for count in Counter(labels).values()]
    # given a list of class probabilities, compute the entropy
    return sum(-p * math.log(p, 2) for p in class_probabilities if p)

def partition_entropy(subsets):
    """find the entropy from this partition of data into subsets"""
    total_count = sum(len(subset) for subset in subsets)
    return sum( entropy(subset) * len(subset) / total_count
                for subset in subsets )
    
def partition_by(inputs, key):
    """returns a dict of inputs partitioned by the key
    each input is a pair (attribute_dict, label)"""
    groups = defaultdict(list)
    # a python dictionary throws a KeyError if you try to get an item 
    # with a _key that is not currently in the dictionary. 
    # The defaultdict() in contrast will simply create any items 
    # that you try to access if they do not exist yet. 
    # To create such a "default" item, it calls the function object 
    # that you pass to the constructor, list() in our case.
    for input in inputs:
        _key = input[0][key]
        groups[_key].append(input)
    return groups   

def partition_entropy_by(inputs, key):
    """computes the entropy corresponding to the given partition"""        
    partitions = partition_by(inputs, key)
    return partition_entropy(partitions.values())        


In [4]:
def build_tree_id3(inputs, split_candidates=None):
    # Quinlan, J. R. 1986. Induction of Decision Trees. Mach. Learn. 1, 1 (Mar. 1986), 81–106
    # https://hunch.net/~coms-4771/quinlan.pdf

    # if this is our first pass, 
    # all keys of the first input are split candidates
    if split_candidates is None:
        split_candidates = inputs[0][0].keys()

    # count Trues and Falses in the inputs
    num_inputs = len(inputs)
    num_trues = len([label for item, label in inputs if label])
    num_falses = num_inputs - num_trues
    
    if num_trues == 0:                  # if only Falses are left
        return False                    # return a "False" leaf
        
    if num_falses == 0:                 # if only Trues are left
        return True                     # return a "True" leaf

    if not split_candidates:            # if no split candidates left
        return num_trues >= num_falses  # return the majority leaf
                            
    # otherwise, split on the best attribute
    best_attribute = min(split_candidates,
                         key=partial(partition_entropy_by, inputs))

    partitions = partition_by(inputs, best_attribute)
    new_candidates = [a for a in split_candidates 
                      if a != best_attribute]
    
    # recursively build the subtrees
    subtrees = { attribute : build_tree_id3(subset, new_candidates)
                 for attribute, subset in partitions.items() }

    subtrees[None] = num_trues > num_falses # default case

    return (best_attribute, subtrees)

In [5]:
def classify(tree, input):
    """classify the input using the given decision tree"""
    
    # if this is a leaf node, return its value
    if tree in [True, False]:
        return tree
   
    # otherwise find the correct subtree
    attribute, subtree_dict = tree
    
    subtree_key = input.get(attribute)  # None if input is missing attribute

    if subtree_key not in subtree_dict: # if no subtree for key,
        subtree_key = None              # we'll use the None subtree
    
    subtree = subtree_dict[subtree_key] # choose the appropriate subtree
    return classify(subtree, input)     # and use it to classify the input


In [7]:
data = pd.read_csv("sample_data/hr_train_1.csv", index_col=0)
ignore = list(data["ignore"])
data = data.drop("ignore", axis=1).to_dict(orient="records")
inputs = []
for i in range(len(ignore)):
  inputs.append((data[i], ignore[i]))

print('Partition entropy by key (lower is better)')
for key in ['level','lang','tweets','phd']:
    print (f'{key:<10}', f'{partition_entropy_by(inputs, key):.3f}')
print()

senior_inputs = [(input, label)
                 for input, label in inputs if input["level"] == "Senior"]

print('Partition entropy by key for Seniors (lower is better)')
for key in ['lang', 'tweets', 'phd']:
    print (f'{key:<10}', f'{partition_entropy_by(senior_inputs, key):.3f}')
print()

print ("building the tree")
tree = build_tree_id3(inputs)
pp (tree)
print('\n', "-"*6, 'TEST', "-"*6)
print ("Senior / Java / tweets / no phd")
print (classify(tree, 
                {"level"  : "Senior", 
                "lang"   : "Java", 
                "tweets" : "yes", 
                "phd"    : "no"})
      ) 

print ("Senior / Python / no tweets / phd")
print (classify(tree, 
                {"level"  : "Senior", 
                  "lang"   : "Python", 
                  "tweets" : "no", 
                  "phd"    : "yes"})
      )

print ("Mid / Java / tweets / no phd")
print (classify(tree, 
                {"level"  : "Mid", 
                  "lang"   : "Java", 
                  "tweets" : "yes", 
                  "phd"    : "no"})
      )

print ("Junior / Python / tweets / no phd")
print (classify(tree, 
                {"level"  : "Junior", 
                  "lang"   : "Python", 
                  "tweets" : "yes", 
                  "phd"    : "no"})
      )

Partition entropy by key (lower is better)
level      0.805
lang       0.864
tweets     0.562
phd        0.855

Partition entropy by key for Seniors (lower is better)
lang       0.928
tweets     0.530
phd        0.961

building the tree
('tweets',
 {None: True,
  'no': ('phd',
         {None: True,
          'no': True,
          'yes': ('level',
                  {None: True,
                   'Junior': True,
                   'Mid': True,
                   'Senior': ('lang',
                              {None: True,
                               'Java': True,
                               'Python': False,
                               'R': False})})}),
  'yes': ('level',
          {None: False,
           'Junior': ('lang',
                      {None: True, 'Java': True, 'Python': True, 'R': False}),
           'Mid': ('phd',
                   {None: False,
                    'no': ('lang',
                           {None: False,
                            'Java': False,


**Person 1:** False because tweets -> Senior -> Java -> no phd -> Still False (strangely having a phd would equal True)

**Person 2:** False because no tweets -> phd -> Senior -> Python -> Still False (None or Java would have equaled True)

**Person 3:** False because tweets -> Mid -> no phd -> Java -> Still False (R would have equaled True)

**Person 4:** True because tweets -> Junior -> Python -> True (Only knowing R could have saved person 4 at this point) :(

# **Part 2**


In [35]:
data = pd.read_csv("sample_data/hr_train_1.csv", index_col=0)
data["gender"] = np.random.randint(0,3,data.shape[0])
count = 0
for i in range(len(data)):
  if data.gender[i] == 2:
    if count < 2:
      data.ignore[i] = True
      count = count + 1
    else:
      data.ignore[i] = False
      count = 0

ignore = list(data["ignore"])
data = data.drop("ignore", axis=1).to_dict(orient="records")
inputs = []
for i in range(len(ignore)):
  inputs.append((data[i], ignore[i]))

print('Partition entropy by key (lower is better)')
for key in ['level','lang','tweets','phd', 'gender']:
    print (f'{key:<10}', f'{partition_entropy_by(inputs, key):.3f}')
print()

senior_inputs = [(input, label)
                 for input, label in inputs if input["level"] == "Senior"]

print('Partition entropy by key for Seniors (lower is better)')
for key in ['lang', 'tweets', 'phd', 'gender']:
    print (f'{key:<10}', f'{partition_entropy_by(senior_inputs, key):.3f}')
print()

print ("building the tree")
tree = build_tree_id3(inputs)
pp (tree)
print('\n', "-"*6, 'TEST', "-"*6)
print ("Senior / R / tweets / phd / gender 2")
print (classify(tree, 
                {"level"  : "Senior", 
                 "lang"   : "R", 
                 "tweets" : "yes", 
                 "phd"    : "yes",
                 "gender" : 2})
      ) 

print ("Senior / R / tweets / phd / gender 1")
print (classify(tree, 
                {"level"  : "Senior", 
                 "lang"   : "R", 
                 "tweets" : "yes", 
                 "phd"    : "yes",
                 "gender" : 1})
      )

print ("Junior / R / tweets / no phd / gender 0")
print (classify(tree, 
                {"level"  : "Junior", 
                 "lang"   : "R", 
                 "tweets" : "yes", 
                 "phd"    : "no",
                 "gender" : 0})
      )

Partition entropy by key (lower is better)
level      0.874
lang       0.888
tweets     0.763
phd        0.885
gender     0.887

Partition entropy by key for Seniors (lower is better)
lang       0.944
tweets     0.751
phd        0.952
gender     0.952

building the tree
('tweets',
 {None: True,
  'no': ('gender',
         {None: True,
          0: True,
          1: ('phd',
              {None: True,
               'no': True,
               'yes': ('level',
                       {None: True,
                        'Junior': True,
                        'Mid': True,
                        'Senior': ('lang',
                                   {None: True,
                                    'Java': True,
                                    'Python': False,
                                    'R': False})})}),
          2: ('lang',
              {None: True,
               'Java': ('level',
                        {None: True,
                         'Junior': ('phd',
              

**Conclusion:** Person 1 is True and is discriminated against based on gender. 

**Person 2** has all the same qualifications, but different gender, and is False.

**Person 3** acts as a control, as this person is False even with worse qualifications than person 1